<a href="https://colab.research.google.com/github/isabel-mm/stylo-r-novels/blob/main/Extracci%C3%B3n_de_palabras_clave.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Código para generar 5 palabras clave de cada texto del corpus 😎**


In [ ]:
import os
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer

# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Cargar modelo de spaCy en la lengua que queramos (en este caso, inglés)
nlp = spacy.load('en_core_web_sm')
nlp.max_length=1880820

# Directorio que contiene los archivos .txt en Google Drive
corpus_dir = '/content/drive/My Drive/Colab Notebooks/inglés'

# Lista de nombres de archivo en el directorio
file_names = os.listdir(corpus_dir)

# Lista para almacenar el contenido de cada archivo
corpus = []

# Lee cada archivo en el directorio y agrega su contenido a la lista "corpus"
for file_name in file_names:
    with open(os.path.join(corpus_dir, file_name), 'r', encoding="latin1") as f:
        corpus.append(f.read())

# Procesar el corpus para eliminar nombres propios y nombres de lugares
processed_corpus = []
for document in corpus:
    doc = nlp(document)
    # Filtrar los tokens que no son nombres propios ni nombres de lugares
    filtered_text = ' '.join(token.text for token in doc if token.ent_type_ not in ('PERSON', 'GPE')) # Etiquetado de entitades en spaCy
    processed_corpus.append(filtered_text)

# Procesa el corpus con spaCy para la lematización y limpieza de stopwords y caracteres no alfabéticos
processed_corpus_lemmatized = []
for document in processed_corpus:
    doc = nlp(document)
    processed_text = ' '.join(token.lemma_ for token in doc if not token.is_stop and token.is_alpha)
    processed_corpus_lemmatized.append(processed_text)

# Creación del objeto TfidfVectorizer con los parámetros deseados
vectorizer = TfidfVectorizer(max_df=0.8, min_df=2)

# Ajuste y transformación del corpus en vectores TF-IDF
tfidf_matrix = vectorizer.fit_transform(processed_corpus_lemmatized)

# Obtiene los nombres de las características
feature_names = vectorizer.get_feature_names_out()

# Función para obtener las palabras clave con mayor puntaje TF-IDF de un documento
def get_top_tfidf(doc_id, feature_names, tfidf_matrix, top_n=5):
    # Obtiene los índices de las características que tienen un valor TF-IDF no nulo en el documento dado
    feature_index = tfidf_matrix[doc_id,:].nonzero()[1]
    # Obtiene los puntajes TF-IDF correspondientes a las características seleccionadas
    tfidf_scores = zip(feature_index, [tfidf_matrix[doc_id, x] for x in feature_index])
    # Ordena las características por puntaje TF-IDF en orden descendente y toma las primeras 'top_n'
    top_items = sorted(tfidf_scores, key=lambda x: -x[1])[:top_n]
    # Inicializa listas para almacenar las palabras clave y sus puntajes TF-IDF
    keywords = []
    scores = []
    # Recorre los elementos seleccionados y almacena las palabras clave y sus puntajes TF-IDF correspondientes
    for item in top_items:
        keywords.append(feature_names[item[0]])
        scores.append(item[1])
    # Devuelve las palabras clave y sus puntajes TF-IDF
    return keywords, scores

In [ ]:
# Define la ruta completa donde deseas guardar el archivo Excel
excel_file_path = '/content/drive/My Drive/Colab Notebooks/palabras_clave.xlsx'

# Crea una lista de diccionarios con los nombres de los archivos y sus palabras clave
data = []
for i, file_name in enumerate(file_names):
    keywords, scores = get_top_tfidf(i, feature_names, tfidf_matrix)
    if keywords:  # Verifica si keywords no está vacío
        row = {'Documento': file_name}
        for j, keyword in enumerate(keywords):
            row[f'Palabra clave {j+1}'] = keyword
        data.append(row)

# Crea un DataFrame a partir de la lista de diccionarios
df = pd.DataFrame(data)

# Guarda el DataFrame en un archivo Excel en la ubicación especificada
df.to_excel(excel_file_path, index=False)

print("¡Terminado! Ya tienes el Excel listo :-)")


¡Terminado! Ya tienes el Excel listo :-)
